In [ ]:
%reset -f
import numpy as np
import scipy
from scipy import stats
from glob import glob
import nibabel as nib
import os
#  Perform a conjunction analysis

## Reference
# Satrajit Ghosh
# 2005_Nichols_ValidConjunctionInferenceMinimumStatistic.pdf
# http://stattrek.com/regression/slope-test.aspx?Tutorial=AP
# http://nilearn.github.io/manipulating_images/manipulating_images.html


## About
#  Greg Ciccarelli
#  October 6, 2016



In [ ]:
def get_t_min(t1,t2):
    # t1 and t2 are t_maps for the two contrasts
    # Informally, in words:  
    # If activations have opposite sign, return 0
    # If activations have the same sign, t_min is the t value closest to zero (keeping the sign)


    # First term:  assume both positive, otherwise 0
    # Second term:  assume both negative, otherwise 0
    t_min = np.maximum(np.minimum(t1,t2), 0) + np.minimum(np.maximum(t1,t2), 0)

    return t_min

In [ ]:
# For a single subject, use the t-map for contrast 1 and contrast 2 to make a t_min map
# Perform significance testing on this map, using multiple hypothesis corrections

# Test data
c1 = np.array([[1,2,3.1], [4, -2, 5]])
c2 = np.array([[3,-1, 10], [3.8, -8, 0]])

c1 = np.random.randn(2,2,3)
c2 = np.random.randn(2,2,3)
print c1
print c2

file_path_sub_list1 = glob(
    os.path.join('/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005', 
                 'sub-voice8*/tstats/tstat01.nii.gz'))
file_path_sub_list2 = glob(
    os.path.join('/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005', 
                 'sub-voice8*/tstats/tstat02.nii.gz'))

In [ ]:
file_path_sub_list1 = glob(
    os.path.join('/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005', 
                 'sub-voice8*/zstats/mni/zstat01.nii.gz'))
file_path_sub_list2 = glob(
    os.path.join('/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005', 
                 'sub-voice8*/zstats/mni/zstat02.nii.gz'))

In [ ]:



# CRUCIAL:  sort to allow pairwise matching
file_path_sub_list1 = sorted(file_path_sub_list1)
file_path_sub_list2 = sorted(file_path_sub_list2)


In [ ]:
print file_path_sub_list1

/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005/sub-voice852/tstats/tstat01.nii.gz
/om/project/voice/processedData/l1analysis/l1output_2016100414/model126/task005/sub-voice852/tstats/tstat02.nii.gz

In [ ]:
# Assume that both contrasts exist for each subject


In [ ]:
img = nib.load(file_path_sub_list1[0])
# get data as numpy array
t1 = img.get_data()

t_min_all = np.empty((np.shape(t1)[0],np.shape(t1)[1],np.shape(t1)[2],0))

print np.shape(t_min_all)

In [ ]:
for f1, f2 in zip(file_path_sub_list1, file_path_sub_list2)[:3]:
    img = nib.load(f1)
    # get data as numpy array
    t1 = img.get_data()
    
    img = nib.load(f2)
    # get data as numpy array
    t2 = img.get_data()    
    
    t_min = get_t_min(t1,t2)
    t_min_all = np.concatenate((t_min_all, t_min[:, :, :, np.newaxis]), axis = 3)

In [ ]:
print np.shape(t_min)
print np.shape(t_min_all)

In [ ]:
# The nichol's paper is organized around maps that already have been thresholded
# into binary form to either declare an effect present for a voxel (H=1) or not (H=0)

#  At the group level, a t-test isn't valid because the inputs to the
#  test are 0 or 1 from each subject, not a continuous value 
#  The null hypothesis then should be derived from a binomial distribution
#  Unfortunately, an arbitrary value must be chosen to be used for the 
#  probability of a 1 when no effect is present.  Otherwise, zero probability is 
#  assigned to presence of one or more "1"s returned from the subjects.
#  Actually the probability of a false positive for a subject should be the error rate e.g. 0.05!


In [ ]:
# Can't perform a t-test conjunction on one subject
#scipy.stats.ttest_1samp(np.array([[1]]), 0, axis=0)

In [ ]:
# t_min_all = np.random.randn(2,2,3,10)
# Convert t values to p values

In [ ]:

t1samp, p1samp = scipy.stats.ttest_1samp(t_min_all, 0, axis=3)

print t1samp
print p1samp[np.logical_not(np.isnan(p1samp))]

In [ ]:
# Using Nichols family wise error correction (Sidak procedure)
# https://en.wikipedia.org/wiki/Family-wise_error_rate#The_.C5.A0id.C3.A1k_procedure

#  Too conservative as hypotheses can only be tested when the tmap is not NaN
V = np.size(t_min)
print V
V = np.sum(np.logical_not(np.isnan(t1samp)))
print V
#  This is a p value
alpha0 = 0.05
alpha_c_fwe = 1 - (1 - alpha0)**(1./V)

print alpha_c_fwe
# Family wise error correction to p value

In [ ]:
# To analyze multiple subjects, for each subject contrast 1 and contrast 2, create a t_min map
# Then, perform a 1 sample t-test (for every voxel independently) across all the t_min maps from all the subjects
# Null hypothesis in all cases is that the value is 0

In [ ]:
# http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html

df = np.shape(t_min_all)[3] - 1
t_thresh_c_fwe = scipy.stats.t.ppf(alpha_c_fwe/2, df, loc=0, scale=1)
print t_thresh_c_fwe

In [ ]:

# Write out the nifti brain with values for visualization


In [ ]:

# Write out a new image using that numpy data and the original affine transformation matrix
imgN = nib.Nifti1Image(t1samp, img.affine)

In [ ]:
nib.save(img, 'test3d.nii.gz')

# Scratch Work

In [ ]:
idx = np.where(np.logical_not(np.isnan(p1samp)))

print np.shape(idx)


In [ ]:
idx = np.asarray(idx)
print np.shape(idx)

In [ ]:
idx[:,0]

In [ ]:
i = 2323

print p1samp[idx[:,i][0], idx[:,i][1], idx[:,i][2]]
print t1samp[idx[:,i][0], idx[:,i][1], idx[:,i][2]]

In [ ]:
df = np.shape(t_min_all)[3] - 1
scipy.stats.t.ppf(0.630136084224/2, df, loc=0, scale=1)